# About: Moodleコンテナの起動

---

Moodleコンテナを起動します。

## 全体構成

![構成](images/moodle-021-01.png)

構築するMoodle環境は次の２つのDockerコンテナで構成されます。

* [Moodle](https://harbor.vcloud.nii.ac.jp/harbor/projects/2/repositories/vcp%2Fmoodle-simple)
  - MoodleのPHPスクリプトを実行するコンテナ
* [MariaDB](https://hub.docker.com/_/mariadb/)
  - Moodleの設定を保存するデータベース
  
構築環境となるノードを事前に「011-VCノードの作成」、「012-EC2インスタンスの作成」、「013-Azure仮想マシンの作成」のいずれかのNotebookで作成しておく必要があります。

## 準備

### Group名の指定

Moodle環境の構築対象を指定するために Ansible のグループ名を指定します。

ノード作成をどのNotebookで行ったかによって、ここで指定する値は異なります。各Notebookでどの値がAnsibleのグループ名に対応するのかを以下に示します。

* 011-VCノードの作成.ipynb
    - UnitGroup名: `ugroup_name`
* 012-EC2インスタンスの作成.ipynb
    - EC2インスタンスの名前: `aws_ec2_name`
* 013-Azure仮想マシンの作成.ipynb
    - 仮想マシンの名前: `azure_vm_name`

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

### 前提条件の確認

このNotebookを実行するための前提条件を満たしていることを確認します。

前提となる条件を以下に示します。

* Ansibleから操作可能であること
* Ansibleから管理者権限でコマンドを実行できること
* Docker が利用可能なこと
* docker compose コマンドがインストールされていること
* CentOS 7であること

対象となるホストにAnsibleで到達可能なことを確認します。

In [ ]:
!ansible {target_group} -m ping

管理者権限でコマンド実行可能なことを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {target_group} -b -a 'whoami'

Dockerが利用可能なことを確認します。

In [ ]:
!ansible {target_group} -a 'docker info'

docker composeコマンドがインストールされていることを確認します。

In [ ]:
!ansible {target_group} -a 'docker compose version'

構築対象のノードが CentOS 7 であること確認します。

In [ ]:
import json

out = !ansible {target_group} -m setup
try:
    idx = [i for i, x in enumerate(out) if x.endswith('| SUCCESS => {')][0]
    data = json.loads(' '.join(['{'] + out[(idx + 1):]))
    distribution = data['ansible_facts']['ansible_distribution']
    distribution_major_version = data['ansible_facts']['ansible_distribution_major_version']
    if distribution != 'CentOS' or distribution_major_version != '7':
        raise RuntimeError(f"ERROR: {distribution} {distribution_major_version}")
except:
    for line in out:
        print(line)
    raise RuntimeError("error!")

## パラメータ設定

### Moodle
Moodleに関するパラメータを指定します。

![Moodle設定](images/moodle-021-02.png)

#### Moodleのコンテナイメージ

Moodleコンテナのイメージ名を指定してください。

ここで指定する値は `harbor.vcloud.nii.ac.jp` にあらかじめ用意してある Moodle イメージから選択する必要があります。次のセルを実行すると選択可能な値の一覧を表示します。

In [ ]:
import requests

url = f'https://harbor.vcloud.nii.ac.jp/api/repositories/vcp/moodle-simple/tags'
res = requests.get(url)
for x in sorted([f"harbor.vcloud.nii.ac.jp/vcp/moodle-simple:{x['name']}"
                 for x in res.json()
                 if not x['name'].endswith('-ssl')], reverse=True):
    print(x)

コンテナイメージのタグによって、どのリリースに対応するMoodle環境を構築するかを指定できます。`3.x.x` のようなタグはマイナーリリースまで特定したコンテナイメージとなります。また `3.x` のようなタグは各ブランチにおける最新のマイナーリリースを意味しています。Moodleのリリース状況に関しては[Moodle - Releases](https://docs.moodle.org/dev/Releases)を参照してください。現在の最新のLTS(Long Term Support)はMoodle 3.9となっています。

In [ ]:
# (例)
# moodle_image_name = 'harbor.vcloud.nii.ac.jp/vcp/moodle-simple:3.9.15'
# moodle_image_name = 'harbor.vcloud.nii.ac.jp/vcp/moodle-simple:3.9'

moodle_image_name = 'harbor.vcloud.nii.ac.jp/vcp/moodle-simple:3.9.15'

#### Moodleの管理者ユーザ名

Moodleの管理者ユーザ名を指定します。

> Moodleのユーザ名に指定できる文字は、小文字英数字と`_`, `-`, `@`, `.`です。

In [ ]:
# (例)
# moodle_admin_name = 'admin'

moodle_admin_name = 

#### Moodleの管理者パスワード

管理者パスワードを指定します。

次のセルを実行するとパスワード入力用の枠が表示されます。管理者パスワードを入力してください（入力後に Enter キーを押すことで入力が完了します）。

> パスワードの値は `admin` 以外の値を指定してください。

In [ ]:
from getpass import getpass
moodle_admin_password = getpass()

#### MoodleのURL

MoodleではサイトのURLを`config.php`の`$CFG->wwwroot`に設定する必要があります。構築対象のURLを指定してください。

In [ ]:
# (例)
# moodle_url = 'http://moodle.example.org'
# moodle_url = 'http://172.30.2.100'

moodle_url = 

#### リバースプロキシ

構築したMoodle環境に対して、一般利用者からのアクセスがリバースプロキシを経由して行われる場合は `moodle_reverseproxy` の値に `True` を設定してください。リバースプロキシを経由せずに構築環境に直接アクセスする構成をとる場合は `False` を設定してください。

In [ ]:
# (例)
# moodle_reverseproxy = False    # リバースプロキシを経由しない場合
# moodle_reverseproxy = True     # リバースプロキシを経由する場合

moodle_reverseproxy = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。ただしパスワードなどの秘匿情報についてはファイルへの保存を行いません。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py
check_parameters(
    'moodle_admin_name',
    'moodle_url',
    nb_vars=locals(),
)

パラメータの値を group_vars ファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    moodle_image_name=moodle_image_name,
    moodle_url=moodle_url,
    moodle_reverseproxy=moodle_reverseproxy,
)

### データベース

Moodleの設定値などを保存するデータベースに関するパラメータを指定します。

![データベース設定](images/moodle-021-03.png)

#### データベースのコンテナイメージ


データベースコンテナのイメージ名を指定してください。このNotebookが構築する環境では MariaDBのコンテナイメージを指定することができます。

In [ ]:
db_image_name = 'mariadb:10.8'

Moodleの設定ファイル`config.php`の中で `$CFG->dbtype` に指定するデータベースの種別を指定してください。

> このNotebookでは `mariadb` を指定した場合の動作確認のみ行っています。

In [ ]:
# (例)
# db_type = 'mariadb'    # MariaDB
# db_type = 'mysql'      # MySQL

db_type = 'mariadb'

#### データベース名

Moodleが使用するデータベース名を指定してください。

In [ ]:
# (例)
# db_moodle_db = 'moodle'

db_moodle_db = 'moodle'

#### データベースの接続ユーザ

Moodleのデータベースに接続するためのデータベースのユーザ名を指定してください。

In [ ]:
# (例)
# db_moodle_db_user = 'moodle'

db_moodle_db_user = 'moodle'

#### データベースのパスワード


Moodleのデータベースに接続するためのパスワードを指定します。

In [ ]:
from getpass import getpass
db_moodle_db_password = getpass()

#### データベースの管理者パスワード

管理者のパスワードを指定します。

In [ ]:
db_root_password = getpass()

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。ただしパスワードなどの秘匿情報についてはファイルへの保存を行いません。

In [ ]:
update_group_vars(
    target_group,
    db_image_name=db_image_name,
    db_type=db_type,
    db_moodle_db=db_moodle_db,
)

## 設定ファイルの配置

Moodleコンテナ、DBコンテナを実行するのに必要となる設定ファイルを構築環境に配置します。

![設定ファイルの配置](images/moodle-021-04.png)

### docker-compose.yml の配置

このNotebookで構築するMoodle環境は複数のコンテナで構成されています。複数コンテナの管理を容易にするために`docker-compose`を利用します。YAMLで記述した設定ファイル`docker-compose.yml`にコンテナ構成を記述することで複数のコンテナの起動、停止などが行えます。

ここでは`docker-compose.yml`を構築環境に配置します。

まず`docker-compose.yml`などを格納するディレクトリ`/srv/moodle`を構築環境に作成します。

In [ ]:
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle state=directory owner={{{{ansible_ssh_user}}}}'

作成したディレクトリに `docker-compose.yml`を配置します。

> ここで配置する `docker-compose.yml` はコンテナ内から設定ファイルをコピーするための一時的なものです。実運用に用いる `docker-compose.yml` は次章で改めて配置します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
import yaml

params = {
    'moodle_admin_name': moodle_admin_name,
    'moodle_admin_password': moodle_admin_password,
    'db_moodle_db_user': db_moodle_db_user,
    'db_moodle_db_password': db_moodle_db_password,
    'db_root_password': db_root_password,
}
with TemporaryDirectory() as workdir:
    vars_path = Path(workdir) / 'moodle.yml'
    with vars_path.open(mode='w') as f:
        yaml.safe_dump(params, f)
    !ansible {target_group} -m template \
        -e prepare=true -e @{vars_path} \
        -a 'src=template/docker-compose.yml dest=/srv/moodle/'

### 各コンテナの設定ファイルを配置する

このNotebookで構築する環境では、コンテナ内で実行する Apache HTTP server の`httpd.conf`などの設定ファイルを構築環境のホスト側に配置しています。これは各コンテナイメージに状態を持たせないようにして、バックアップ、リストアなどを容易に行うための対応策です。ホスト側に配置した設定ファイルは`docker-compose.yml`で指定した[bind mount](https://docs.docker.com/storage/bind-mounts/)の機能を利用してコンテナから参照できるようにしています。

bind mountによって配置する設定ファイルのホスト環境におけるパスとコンテナ環境におけるパスの対応関係を以下に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ名</th>
    <th style="text-align:left;">コンテナ環境のパス</th>
    <th style="text-align:left;">ホスト環境のパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">moodle</td>
    <td style="text-align:left;">/etc/httpd/conf/httpd.conf</td>
    <td style="text-align:left;">/srv/moodle/moodle/conf/httpd/conf/httpd.conf</td>
  </tr>
  <tr>
    <td style="text-align:left;">moodle</td>
    <td style="text-align:left;">/etc/httpd/conf.d/</td>
    <td style="text-align:left;">/srv/moodle/moodle/conf/httpd/conf.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">moodle</td>
    <td style="text-align:left;">/etc/httpd/conf.modules.d/</td>
    <td style="text-align:left;">/srv/moodle/moodle/conf/httpd/conf.modules.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">moodle</td>
    <td style="text-align:left;">/etc/php.ini</td>
    <td style="text-align:left;">/srv/moodle/moodle/conf/php.ini</td>
  </tr>
  <tr>
    <td style="text-align:left;">moodle</td>
    <td style="text-align:left;">/etc/php.d/</td>
    <td style="text-align:left;">/srv/moodle/moodle/conf/php.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">db</td>
    <td style="text-align:left;">/etc/mysql/mariadb.conf.d/</td>
    <td style="text-align:left;">/srv/moodle/db-0/conf/mariadb.conf.d/</td>
  </tr>
</table>

この節では、コンテナイメージの設定ファイルをコピーして構築環境のホスト側への配置を行います。

コンテナ内の設定ファイルをホスト側にコピーするために、一時的にコンテナを起動します。まず、利用するコンテナイメージを取得します。

In [ ]:
!ansible {target_group} -a \
    'chdir=/srv/moodle docker compose pull'

設定ファイルをコピーするために一時的なコンテナ起動を行います。

In [ ]:
!ansible {target_group} -a \
    'chdir=/srv/moodle docker compose up -d'

コンテナから設定ファイルなどのコピーを行います。

> 不整合をさけるために、ホスト側に配置されているファイルを全て削除してからコピーを行います。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    mkdir -p db/data db/misc moodle/conf moodle/data/moodledata'
!ansible {target_group} -b -m shell -a 'chdir=/srv/moodle \
    rm -rf db/conf moodle/data/php moodle/conf/*'
!ansible {target_group} -m shell -a 'chdir=/srv/moodle \
    docker cp db:/etc/mysql db/conf; \
    docker cp moodle:/etc/httpd moodle/conf; \
    docker cp moodle:/etc/php.ini moodle/conf; \
    docker cp moodle:/etc/php.d moodle/conf; \
    docker cp moodle:/var/www/moodle moodle/data/php'

設定ファイルのコピーが完了したので、一時的に起動したコンテナを停止、削除します。

In [ ]:
!ansible {target_group} -a \
    'chdir=/srv/moodle docker compose down'

### ログ関連の準備

コンテナで実行するサービスのログに関するセットアップを行います。

#### ログ出力先のディレクトリの作成

このNotebookで構築する環境では、コンテナ内で実行する Apache HTTP server などのログ出力先ディレクトリをホスト側のディレクトリに[bind mount](https://docs.docker.com/storage/bind-mounts/)します。これによりホスト側からもログの参照が容易になります。

ログ出力先ディレクトリのコンテナ環境とホスト環境での対応関係を以下に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ名</th>
    <th style="text-align:left;">コンテナ環境のパス</th>
    <th style="text-align:left;">ホスト環境のパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">moodle</td>
    <td style="text-align:left;">/var/log/httpd</td>
    <td style="text-align:left;">/var/log/httpd</td>
  </tr>
  <tr>
    <td style="text-align:left;">db</td>
    <td style="text-align:left;">/var/log/mysql</td>
    <td style="text-align:left;">/var/log/mysql</td>
  </tr>
</table>

ログ出力先のディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a 'path=/var/log/httpd state=directory'
!ansible {target_group} -b -m file -a 'path=/var/log/mysql owner=999 group=adm state=directory'

#### logrotateの設定ファイルを配置する

Moodleコンテナ、DBコンテナのログをローテーションするための設定ファイルを配置します。logrotateはホスト環境で実行するので、ホスト側の `/etc/logrotate.d/` に配置します。 

In [ ]:
!ansible {target_group} -b -m copy -a 'src=template/logrotate.d/httpd dest=/etc/logrotate.d/'
!ansible {target_group} -b -m copy -a 'src=template/logrotate.d/mysql-server dest=/etc/logrotate.d/'

#### mysqladmin の設定ファイルを配置する

MariaDB/MySQL に対して logrotate を行うには管理者権限で `mysqladmin flush-logs` などのコマンドを実行出来るようにする必要があります。これを可能にするために管理者パスワードの情報を `/root/.my.cnf` に格納します。

コンテナに状態を持たせないようにするために設定ファイルはホスト環境に配置したものを bind mountすることにします。コンテナ環境、ホスト環境におけるパスを以下に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ環境のパス</th>
    <th style="text-align:left;">ホスト環境のパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/root/.my.cnf</td>
    <td style="text-align:left;">/srv/moodle/db/misc/my.cnf</td>
  </tr>
</table>

設定ファイルをホスト側に配置します。

In [ ]:
with TemporaryDirectory() as workdir:
    vars_path = Path(workdir) / 'moodle.yml'
    with vars_path.open(mode='w') as f:
        yaml.safe_dump(params, f)
    !ansible {target_group} -b -m template \
        -e prepare=true -e @{vars_path} \
        -a 'src=template/mysql/my.cnf dest=/srv/moodle/db/misc/ mode=0600 owner=root group=root'

#### MariaDB 10.6 の設定ファイルを配置する

MariaDB 10.6 から導入されたシステム変数に [innodb_read_only_compressed](https://mariadb.com/kb/en/innodb-system-variables/#innodb_read_only_compressed) があります。この値のデフォルト値は10.6.0から10.6.5 まで ON となっています。その場合Moodleのセットアップ時のテーブル操作がエラーとなってしまいます。これを回避するために `innodb_read_only_compressed` を `OFF`にする設定ファイルを配置します。設定は`[mariadb-10.6]`グループで行いMariaDB 10.6以外には影響が無いようにします。

設定ファイルのコンテナ環境、ホスト環境におけるパスを以下に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ環境のパス</th>
    <th style="text-align:left;">ホスト環境のパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/mysql/mariadb.conf.d/99-local.cnf</td>
    <td style="text-align:left;">/srv/moodle/db/conf/mariadb.conf.d/99-local.cnf</td>
  </tr>
</table>

In [ ]:
!ansible {target_group} -m copy \
    -a 'src=template/mariadb.conf.d/99-local.cnf \
    dest=/srv/moodle/db/conf/mariadb.conf.d/'

## アプリケーションコンテナの起動

Moodleコンテナ、データベースコンテナを起動して、Moodle環境を起動します。

![コンテナの起動](images/moodle-021-05.png)

`docker-compose.yml`を配置します。

> ここで配置する`docker-compose.yml`が実運用で使用するものとなります。前章で配置したものとの違いは設定ファイルに関する bind mount 設定を追加していることです。

In [ ]:
with TemporaryDirectory() as workdir:
    vars_path = Path(workdir) / 'moodle.yml'
    with vars_path.open(mode='w') as f:
        yaml.safe_dump(params, f)
    !ansible {target_group} -m template \
        -e @{vars_path} \
        -a 'src=template/docker-compose.yml dest=/srv/moodle/'

コンテナの起動を行います。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up -d'

Moodleのインストール処理が完了してMoodleのサービスが開始されるまで数分程度を要します。ここではMoodleが開始されるまでの待ち合わせを行います。

In [ ]:
%run scripts/utils.py

def check_http_access():
    !ansible {target_group} -a \
        "curl -s -f -I http://localhost" | grep -w HTTP

retry_exec(check_http_access, err=Exception)

## Moodle を利用できることを確認

構築したMoodle環境にアクセスし、Moodle が利用できることを確認します。

### アドレスの確認

次のセルを実行すると表示されるリンクが、構築したMoodle環境のアドレスです。リンクをクリックしてMoodle環境にログインしてください。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}" target="_blank">{0}</a>'.format(moodle_url))

### ライブログの確認

Moodle のライブログを確認します。次のセルを実行すると表示されるリンクをクリックしてください。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/report/loglive/index.php" target="_blank">{0}/report/loglive/index.php</a>'.format(moodle_url))

## コンテナのログを確認する

MoodleコンテナのApache HTTP Serverのログはホスト環境の `/var/log/httpd/` に出力されます。アクセスログ `access_log` の内容を確認してみます。

In [ ]:
!ansible {target_group} -a 'tail /var/log/httpd/access_log'

DBコンテナのログはホスト環境の `/var/log/mysql/` に出力されます。エラーログ `error.log` の内容を確認してみます。

In [ ]:
!ansible {target_group} -b -a 'tail /var/log/mysql/error.log'

## crontabの設定

Moodleのスケジュールタスクを実行するために必要となるスクリプトを1分毎に実行するように crontab を設定します。

In [ ]:
!ansible {target_group} -m cron -a 'name=PATH env=yes job=/usr/bin:/usr/local/bin'
!ansible {target_group} -m cron -a 'name={target_group} \
    job="docker compose --project-directory=/srv/moodle exec -T moodle /usr/bin/php /var/www/moodle/admin/cli/cron.php > /dev/null"'